## Deliverable 2. Create a Customer Travel Destinations Map.

In [58]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [59]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Right Now,Country,Date
0,0,Gat,31.6100,34.7642,51.40,100,54,9.24,broken clouds,IL,2022-01-31 23:38:57
1,1,Rikitea,-23.1203,-134.9692,77.83,66,73,3.78,broken clouds,PF,2022-01-31 23:40:00
2,2,Necochea,-38.5473,-58.7368,62.04,53,63,8.32,broken clouds,AR,2022-01-31 23:37:48
3,3,Lorengau,-2.0226,147.2712,77.83,90,100,10.31,overcast clouds,PG,2022-01-31 23:41:57
4,4,Vaini,-21.2000,-175.2000,86.16,79,75,10.36,broken clouds,TO,2022-01-31 23:41:57


In [69]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [70]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Right Now,Country,Date
1,1,Rikitea,-23.1203,-134.9692,77.83,66,73,3.78,broken clouds,PF,2022-01-31 23:40:00
3,3,Lorengau,-2.0226,147.2712,77.83,90,100,10.31,overcast clouds,PG,2022-01-31 23:41:57
8,8,Mataura,-46.1927,168.8643,75.47,50,69,2.42,broken clouds,NZ,2022-01-31 23:41:58
13,13,Tefe,-3.3542,-64.7114,80.71,83,20,3.44,few clouds,BR,2022-01-31 23:40:52
15,15,Saint-Philippe,-21.3585,55.7679,78.39,80,100,14.81,overcast clouds,RE,2022-01-31 23:42:00
16,16,Mahebourg,-20.4081,57.7000,80.26,78,75,14.97,broken clouds,MU,2022-01-31 23:42:01
17,17,Puerto Ayora,-0.7393,-90.3518,74.55,93,64,1.99,broken clouds,EC,2022-01-31 23:42:02
18,18,Santiago,-33.4569,-70.6483,78.24,39,0,8.05,clear sky,CL,2022-01-31 23:42:02
20,20,Cape Town,-33.9258,18.4232,74.12,80,0,2.30,clear sky,ZA,2022-01-31 23:41:30
22,22,Atar,20.5169,-13.0499,80.28,13,100,9.91,overcast clouds,MR,2022-01-31 23:42:03


In [71]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID              243
City                 243
Lat                  243
Lng                  243
Max Temp             243
Humidity             243
Cloudiness           243
Wind Speed           243
Weather Right Now    243
Country              241
Date                 243
dtype: int64

In [72]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()


City_ID              241
City                 241
Lat                  241
Lng                  241
Max Temp             241
Humidity             241
Cloudiness           241
Wind Speed           241
Weather Right Now    241
Country              241
Date                 241
dtype: int64

In [73]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City","Max Temp","Country", "Weather Right Now", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Max Temp,Country,Weather Right Now,Lat,Lng,Hotel Name
1,Rikitea,77.83,PF,broken clouds,-23.1203,-134.9692,
3,Lorengau,77.83,PG,overcast clouds,-2.0226,147.2712,
8,Mataura,75.47,NZ,broken clouds,-46.1927,168.8643,
13,Tefe,80.71,BR,few clouds,-3.3542,-64.7114,
15,Saint-Philippe,78.39,RE,overcast clouds,-21.3585,55.7679,
...,...,...,...,...,...,...,...
676,Pointe Michel,77.85,DM,broken clouds,15.2500,-61.3833,
681,Greymouth,72.16,NZ,overcast clouds,-42.4667,171.2000,
683,Pontianak,73.38,ID,mist,-0.0333,109.3333,
686,Muncar,79.16,ID,overcast clouds,-8.4333,114.3333,


In [74]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [75]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                 241
Max Temp             241
Country              241
Weather Right Now    241
Lat                  241
Lng                  241
Hotel Name           241
dtype: int64

In [76]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [77]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [79]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))